In [1]:
import os
import pandas as pd
import shutil
codelist_files_folder="Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added"
filepath_database_to_match="Matching_with_Caliber.csv" #---
filepath_database_our_codelists_folder="Put_Codelist_Files_to_Shrink_from_caliber_here"
filepath_output="Asra_Shrinking_Lists_from_Caliber"
filepath_snomed_from_medcode="Mapping_Multimorbidity_Codelist_16.11.2021.xlsx"

In [2]:
codelist_files = os.listdir(codelist_files_folder)
codelist_files

['axson_asthma_cdd2NMH5QDWVdEfTQNNfpK_ICD10.csv',
 'axson_asthma_cdd2NMH5QDWVdEfTQNNfpK_ICD11.csv',
 'axson_asthma_cdd2NMH5QDWVdEfTQNNfpK_SNOMEDCT.csv',
 'axson_asthma_cdd2NMH5QDWVdEfTQNNfpK_UKBIOBANK.csv',
 'axson_bronchiestasis_ZckoXfUWNXn8Jn7fdLQuxj_ICD10.csv',
 'axson_bronchiestasis_ZckoXfUWNXn8Jn7fdLQuxj_ICD11.csv',
 'axson_bronchiestasis_ZckoXfUWNXn8Jn7fdLQuxj_SNOMEDCT.csv',
 'axson_bronchiestasis_ZckoXfUWNXn8Jn7fdLQuxj_UKBIOBANK.csv',
 'axson_COPD_Y9JxuQRFPprJDMHSPowJYs_ICD10.csv',
 'axson_COPD_Y9JxuQRFPprJDMHSPowJYs_ICD11.csv',
 'axson_COPD_Y9JxuQRFPprJDMHSPowJYs_SNOMEDCT.csv',
 'axson_COPD_Y9JxuQRFPprJDMHSPowJYs_UKBIOBANK.csv',
 'axson_cystic_fibrosis_5eDwF2jVb4XWHMtooSYTbB_ICD10.csv',
 'axson_cystic_fibrosis_5eDwF2jVb4XWHMtooSYTbB_ICD11.csv',
 'axson_cystic_fibrosis_5eDwF2jVb4XWHMtooSYTbB_SNOMEDCT.csv',
 'axson_cystic_fibrosis_5eDwF2jVb4XWHMtooSYTbB_UKBIOBANK.csv',
 'axson_pneumonia_GL46c8PLQMdPuj7QuPtEh9_ICD10.csv',
 'axson_pneumonia_GL46c8PLQMdPuj7QuPtEh9_ICD11.csv',
 'axso

In [3]:
df_to_match = pd.read_csv(filepath_database_to_match) 
df_to_match

,Original Name,Target Name
0,Anaemia B12 Deficiency,kuan oth anaemia and kuan oth haem anaemia and...
1,Anaemia Folate Deficiency,kuan oth anaemia and kuan oth haem anaemia and...
2,Anaemia Haemolytic,kuan oth anaemia and kuan oth haem anaemia and...
3,Anaemia Iron Deficiency,kuan oth anaemia and kuan oth haem anaemia and...
4,Anaemia Other,kuan oth anaemia and kuan oth haem anaemia and...
5,CKD,covid impact ccu00201 CKD
6,Glaucoma,kuan glaucoma
7,Hypertension,kuan hypertension and carr hypertension
8,Sickle cell anaemia,kuan sickle cell BoyuDsY7sCnGANEAFw9ffb and ku...


In [4]:
list_target_names = df_to_match['Target Name'].tolist()
list_target_names 

['kuan oth anaemia and kuan oth haem anaemia and kuan aplastic and kuan folatedef',
 'kuan oth anaemia and kuan oth haem anaemia and kuan aplastic and kuan folatedef',
 'kuan oth anaemia and kuan oth haem anaemia and kuan aplastic and kuan folatedef',
 'kuan oth anaemia and kuan oth haem anaemia and kuan aplastic and kuan folatedef',
 'kuan oth anaemia and kuan oth haem anaemia and kuan aplastic and kuan folatedef',
 'covid impact ccu00201 CKD',
 'kuan glaucoma',
 'kuan hypertension and carr hypertension',
 'kuan sickle cell BoyuDsY7sCnGANEAFw9ffb and kuan sickle cell and kuan sickle trait HpKeYNjMWk7CReFWYJUAZ8']

In [5]:
list_source_names = df_to_match['Original Name'].tolist()
list_source_names 

['Anaemia B12 Deficiency',
 'Anaemia Folate Deficiency',
 'Anaemia Haemolytic',
 'Anaemia Iron Deficiency',
 'Anaemia Other',
 'CKD',
 'Glaucoma',
 'Hypertension',
 'Sickle cell anaemia']

In [6]:
def medcode_to_snomed_converted_file(filepath_snomed_from_medcode, codelist_files_folder, filename):    
    df_snomed = pd.read_excel(filepath_snomed_from_medcode, sheet_name=0, dtype={'SnomedCTConceptId': str, 'SnomedCTDescriptionId': str})
    df_snomed_readcode_mapping=df_snomed[['CleansedReadCode', 'SnomedCTConceptId']]
    #print(df_snomed_medcode_mapping)

    readcode_files_path=codelist_files_folder+"/"+ filename
    df_readcode_codelist = pd.read_csv(readcode_files_path)
    #print(df_medcode_codelist)

    df_readcode_codelist.rename(columns={'Readcode' : 'CleansedReadCode'}, inplace=True) 

    df_map=pd.merge(df_readcode_codelist, df_snomed_readcode_mapping, on='CleansedReadCode', how='left')
    df_map=df_map.drop_duplicates()
    df_map.rename(columns={'SnomedCTConceptId':'SNOMED CT Concept ID'}, inplace = True) #--renaming this as caliber has name 'SNOMED CT Concept ID'---
    #print(df_map)

    snomed_filename = filename.replace("CPRD", "Asra_Converted_SNOMED")
    snomed_created_filepath=codelist_files_folder+"/"+snomed_filename
    print(snomed_created_filepath)
    df_map.to_csv(snomed_created_filepath, index=False)

    return snomed_created_filepath

In [7]:
def find_destination_files_paths(target_name, codelist_files, codelist_files_folder):
    destination_codelist_csv_paths=[]
    part_of_name = target_name.split(" ")
    Found=False
    for search_word in part_of_name:
        if search_word.lower() == 'kuan':
            continue
        for filename in codelist_files:
            search_word_stripped = search_word.strip()
            if search_word_stripped.lower() in filename.lower() and len(search_word_stripped) != 0 and 'and' not in search_word_stripped.lower():
                print("----------Found-------------")
                if 'SNOMEDCT' in filename:
                    print("----------in SNOMEDCT--------------")
                    Found=True
                    one_filepath=codelist_files_folder+"/"+filename
                    print("------------Appending------------")
                    if one_filepath not in destination_codelist_csv_paths:
                        print(one_filepath)
                        destination_codelist_csv_paths.append(one_filepath)
                elif 'CPRD' in filename:
                    print("-----------in CPRD------------")
                    snomed_created_filepath=medcode_to_snomed_converted_file(filepath_snomed_from_medcode, codelist_files_folder, filename)
                    print("------------Appending------------")
                    if snomed_created_filepath not in destination_codelist_csv_paths:
                        print(snomed_created_filepath)
                        destination_codelist_csv_paths.append(snomed_created_filepath)

            # if 'SNOMEDCT' in filename: #---because we want to search only in snomed files----
                
            #     if search_word_stripped.lower() in filename.lower() and len(search_word_stripped) != 0 and 'and' not in search_word_stripped.lower():
            #         print("search word: "+search_word_stripped.lower()+", filename: "+ filename.lower())
                    
                
    # if not Found:
    #     print(target_name, ".")
    #     part_of_name = target_name.split(" ")
    #     for search_word in part_of_name:
    #         print(search_word, ",", search_word.replace(" ", ""), ".")
    #         print(len(search_word))
    #         for filename in codelist_files:
    #             if 'SNOMEDCT' in filename: 
    #                 if search_word_stripped.lower() in filename.lower():
    #                     print("Good")
    #                 else:
    #                     print(search_word.lower(), filename.lower())
    return destination_codelist_csv_paths

# def find_destination_files_paths(target_name, codelist_files, codelist_files_folder):
#     destination_codelist_csv_paths=[]
#     part_of_name = target_name.split(" ")
#     Found=False
#     for search_word in part_of_name:
#         if search_word.lower() == 'kuan':
#             continue
#         for filename in codelist_files:
#             if 'SNOMEDCT' in filename: #---because we want to search only in snomed files----
#                 search_word_stripped = search_word.strip()
#                 if search_word_stripped.lower() in filename.lower() and len(search_word_stripped)is not 0 and 'and' not in search_word_stripped.lower():
#                     print("search word: "+search_word_stripped.lower()+", filename: "+ filename.lower())
#                     Found=True
#                     one_path=codelist_files_folder+"/"+filename
#                     if one_path not in destination_codelist_csv_paths:
#                         destination_codelist_csv_paths.append(one_path)
                
#     return destination_codelist_csv_paths

In [8]:
# match one list to another one by one
source_file_path_list=[]
destination_file_path_list=[]
#temp=0
for source_name in list_source_names:
    #print(source_name)
    # if temp>2:
    #     break
    # temp+=1
    part_of_name = source_name.split(" ")
    found=False
    for search_word in part_of_name:
        print(search_word)
        for target_name in list_target_names:
            if search_word.lower() in target_name.lower():
                print("found")
                found=True
                # print("Found: ", search_word)
                # print("for ", source_name)
                # print("in ", target_name)
                source_codelist_csv_path=filepath_database_our_codelists_folder+"/"+source_name+".csv"
                #print(source_codelist_csv_path)
                source_file_exist=os.path.isfile(source_codelist_csv_path)
                if not source_file_exist:
                    print(source_codelist_csv_path, " does not exist", )
                    found=False
                else:
                    destination_codelist_csv_paths=find_destination_files_paths(target_name, codelist_files, codelist_files_folder)
                    #print(destination_codelist_csv_paths)
                    if len(destination_codelist_csv_paths)==0:
                        print("Destination paths does not exist for target names", target_name, ".")                        
                        found=False
                    else:
                        source_file_path_list.append(source_codelist_csv_path)
                        destination_file_path_list.append(destination_codelist_csv_paths)
            if found:
                break
        if found:
            break 
print("Completed")

Anaemia
found
----------Found-------------
----------Found-------------
----------Found-------------
----------Found-------------
-----------in CPRD------------
Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_cardiomyo_oth_L9Vd69SHmNTPEVacrtJdby_Asra_Converted_SNOMED.csv
------------Appending------------
Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_cardiomyo_oth_L9Vd69SHmNTPEVacrtJdby_Asra_Converted_SNOMED.csv
----------Found-------------
----------Found-------------
----------Found-------------
----------Found-------------
-----------in CPRD------------
Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_oth_anaemia_TkopQyYsqmzwVKiwfAHQnR_Asra_Converted_SNOMED.csv
------------Appending------------
Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_oth_anaemia_TkopQyYsqmzwVKiwfAHQnR_Asra_Converted_SNOMED.csv
----------Found-------------
----------Found-------------
----------Found-------------
----------Found-------------
----------Found-------------
-----------

In [9]:
df_matched_paths = pd.DataFrame({'Source Paths':source_file_path_list, 'Destination Paths': destination_file_path_list}) # df_matched_paths = pd.DataFrame({'Source Paths':source_file_path_list})
df_matched_paths.to_csv('caliber_matched_files.csv', index=False)  
df_matched_paths

,Source Paths,Destination Paths
0,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
1,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
2,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
3,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
4,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
5,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
6,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
7,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...
8,Put_Codelist_Files_to_Shrink_from_caliber_here...,[Caliber/Caliber_Codelist_Files_with_SNOMEDs_A...


# Shrinking

In [10]:
list_source_paths = df_matched_paths['Source Paths'].tolist()
list_destination_paths = df_matched_paths['Destination Paths'].tolist()

In [11]:
list_source_paths 

['Put_Codelist_Files_to_Shrink_from_caliber_here/Anaemia B12 Deficiency.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Anaemia Folate Deficiency.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Anaemia Haemolytic.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Anaemia Iron Deficiency.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Anaemia Other.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/CKD.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Glaucoma.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Hypertension.csv',
 'Put_Codelist_Files_to_Shrink_from_caliber_here/Sickle cell anaemia.csv']

In [12]:
list_destination_paths

[['Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_cardiomyo_oth_L9Vd69SHmNTPEVacrtJdby_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_oth_anaemia_TkopQyYsqmzwVKiwfAHQnR_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_oth_haem_anaemia_6iaKHraVPpnAFrkvTvhSNz_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_pneumothorax_a5NNubakLzYdsQiw9uJPBB_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_pri_mesothelioma_LXgfieFkyhgDuovn87QDeQ_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_pri_other_5oEhmm6bfGj5TwvQWpso7s_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_sec_other_DCDEYHfps3kCT6QfRGVNsx_Asra_Converted_SNOMED.csv',
  'Caliber/Caliber_Codelist_Files_with_SNOMEDs_Added/kuan_sec_oth_thrombocytopaenia_f5FuMcav9uFcEjkbZ5JtG4_Asra_Converted_SNOMED.csv',
  'Calibe

In [13]:
def shrink_file(source_filepath, destination_filepaths, shrinked_filepath):
    df_source_file = pd.read_csv(source_filepath, dtype={'SnomedCTConceptId': str}) 
    #print(df_database_efi['SnomedCTConceptId'])
    list1_source_snomeds = df_source_file['SnomedCTConceptId'].tolist()
    #print(list1_source_snomeds)
    

    list2_destination_snomeds=[]
    for destination_filepath in destination_filepaths: #--goto each path and search for particular snomed-----
        df_destination_file_orig = pd.read_csv(destination_filepath, dtype={'SNOMED CT Concept ID': str}) 
        if "SNOMED CT Concept ID" in df_destination_file_orig.columns: #---to avoid a caliber issues, in caliber files some have missing column 'SNOMED CT Concept ID' 
            df_destination_file = pd.read_csv(destination_filepath, dtype={'SNOMED CT Concept ID': str}) 
            list_one_destination_snomeds = df_destination_file['SNOMED CT Concept ID'].values.tolist()
            # print("For Destination ", destination_filepath)
            # print(list_one_destination_snomeds)
            list2_destination_snomeds.extend(list_one_destination_snomeds)

    #print(list2_destination_snomeds)
    #print("Missing values in first list:", (set(list2_destination_snomeds).difference(list1_source_snomeds))) #-- we dont need it--
    
    additional_snomed_codes=list(set(list1_source_snomeds).difference(list2_destination_snomeds))
    print("Additional values in first list:", additional_snomed_codes)

    if len(additional_snomed_codes)==0:
        print("Congratulations all snomed codes were present for this file: ", source_filepath)
    else:   #----this code shrink files for identified snomed's---
        df_shrinked = pd.read_csv(source_filepath, dtype={'SnomedCTConceptId': str}) 
        #common_snomeds=list(set(list1_source_snomeds)&set(list2_destination_snomeds))
        print(additional_snomed_codes)
        print(len(list1_source_snomeds), len(list2_destination_snomeds), len(additional_snomed_codes))
        common_snomeds=list(set(list1_source_snomeds).difference(additional_snomed_codes))#[value for value in list1_source_snomeds if value in list2_destination_snomeds]
        print("common snomeds: ", common_snomeds)
        for snomed in common_snomeds:
            print(snomed)
            df_shrinked = df_shrinked.drop(df_shrinked[df_shrinked['SnomedCTConceptId'] == snomed].index)
        print("Saving... ",shrinked_filepath )
        print("Deleted rows: ", len(additional_snomed_codes))
        df_shrinked.to_csv(shrinked_filepath, index=False)  

In [14]:
index_list=0
num_shrinked_files=0 #---this is just for reference---
for source_filepath in list_source_paths:
    parts_of_name = source_filepath.split("/")
    source_filename=parts_of_name[-1]
    shrinked_filename="Shrinked "+source_filename
    shrinked_filepath=filepath_output+"/"+shrinked_filename
    # print(source_filepath)
    # print(source_filename)
    # print(shrinked_filename)
    # print(shrinked_filepath)
    destination_filepaths = list_destination_paths[index_list]
    if len(destination_filepaths)==0:
        # print("No Files are Present in Caliber related to this Condition, so not shrinking "+source_filename)
        # print("----Destination Path: ", destination_filepaths)
        shutil.copy(source_filepath, shrinked_filepath)
    else:
        shrink_file(source_filepath, destination_filepaths, shrinked_filepath)
        num_shrinked_files+=1
    index_list+=1

print("Shrinking Completed-----")     
print("Total Number of Shrinked files are "+str(num_shrinked_files)+" out of total files "+str(len(list_source_paths)))     

Additional values in first list: ['234360003', '161457000', '170823005', '49472006', '267517008', '234361004', '191142007', '234363001', '165645009', '190634004', '84027009', '234362006']
['234360003', '161457000', '170823005', '49472006', '267517008', '234361004', '191142007', '234363001', '165645009', '190634004', '84027009', '234362006']
14 730 12
common snomeds:  ['191154007']
191154007
Saving...  Asra_Shrinking_Lists_from_Caliber/Shrinked Anaemia B12 Deficiency.csv
Deleted rows:  12
Additional values in first list: ['191150003', '267518003', '191148006', '191149003']
['191150003', '267518003', '191148006', '191149003']
14 730 4
common snomeds:  ['190633005', '234358000', '15332004', '191154007', '85649008']
190633005
234358000
15332004
191154007
85649008
Saving...  Asra_Shrinking_Lists_from_Caliber/Shrinked Anaemia Folate Deficiency.csv
Deleted rows:  4
Additional values in first list: ['221241000000101', '419211000000105', '26682008', '191218003', '161458005', '191213007', '19728